# Class dunder methods and duck typing

In this tutorial we'll see how we can make supercharge our classes to make them more interoperable with python's built-in functions.

This tutorial is targeted at someone who has some experience working with python classes (i.e. understands concepts such as class/instance attributes/methods, etc.).

## Premise

We want to built a class that represents a regular card deck.

![](https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fclipart-library.com%2Fimages%2FLTd5gKzAc.png)

A deck consists of 52 cards. Each card has a rank (i.e. *2-10, J, Q, K, A*) and a suit (*spade, club, diamond, heart*), where *spades* and *clubs* are *black* in color, while *diamonds* and *hearts* are *red*.

We will represent each card by 2 characters, the first being the rank and the second the suit. E.g. 5 of hearts will be represented as `'5H'`.

Let's first crete a very basic class to contain all possible cards.



In [1]:
class Deck:
  ranks = tuple(range(2, 11)) + ('J', 'Q', 'K', 'A')
  suits = ('D', 'H', 'C', 'S')
  cards = tuple(str(v) + s for v in ranks for s in ('D', 'H', 'C', 'S'))
  colors = {'D': 'R', 'H': 'R', 'C': 'B', 'S': 'B'}

The main class attribute we are interested in is `cards` which is a tuple containing all 52 playing cards.

In [2]:
Deck.cards[:10]

('2D', '2H', '2C', '2S', '3D', '3H', '3C', '3S', '4D', '4H')

## Add state to class

For this deck to be functional it needs to hold some **state**, i.e. if someone draws some cards, which remain in the deck.

Consequently we'll add the following functionality to our class:
- Add a couple of instance variables `stock` and `drawn`; the first is a list of all playing cards currently in the deck, while the second is a list of all cards that have been drawn.
- Rename class attributes from lowercase to upper and add an underscore in the beginning. It is a **convention** in python that **uppercase variables represent constants** (in our case the possible ranks/suits/colors/cards in a deck). Again, by convention, **variables starting with an underscore represent class internals that shouldn't be tampered with** by users of this class.
- Add a method to shuffle the deck.
- Add a method to draw `n` cards from the deck.
- Add a method to *reset* the deck, i.e. reshuffle all 52 cards.
- Add a **constuctor** to the class. The constructor is a special method named `__init__` that is called whenever a new instance is created. We usually use this method to **initialize** an object with it's initial state. In our case this means that we will initialize the `stock` attribute to conain all cards shuffled and `stock` as an empty list.

In [3]:
import numpy as np


class Deck:
    _RANKS = tuple(range(2, 11)) + ('J', 'Q', 'K', 'A')
    _SUITS = ('D', 'H', 'C', 'S')
    _CARDS = tuple(str(v) + s for v in _RANKS for s in ('D', 'H', 'C', 'S'))
    _COLORS = {'D': 'R', 'H': 'R', 'C': 'B', 'S': 'B'}

    def __init__(self, num_packs=1):
        self.stock = self.shuffle(self._CARDS * num_packs)
        self.drawn = []

    def draw(self, n=1):
        cards = [self.stock.pop(0) for _ in range(n)]
        self.drawn += cards
        return cards

    def reset(self):
        self.stock = self.shuffle(self.stock + self.drawn)
        self.drawn = []

    def shuffle(self, cards):
        return list(np.random.permutation(cards))

Let's test our class' basic functionality.

In [4]:
deck = Deck()

print('Initial stock:', deck.stock[:10])
print(f'{len(deck.stock) = }')

drawn_cards = deck.draw(3)
print('Drawn cards:', drawn_cards)

print('Stock after drawing cards:', deck.stock[:10])
print(f'{len(deck.stock) = }')

Initial stock: ['9H', '2D', '9C', '6C', '5D', '6H', 'KD', '10S', 'AC', '10D']
len(deck.stock) = 52
Drawn cards: ['9H', '2D', '9C']
Stock after drawing cards: ['6C', '5D', '6H', 'KD', '10S', 'AC', '10D', '5C', '8C', '7S']
len(deck.stock) = 49


## Use properties to obscure internal instance attributes

Let's think abouth our class a bit more abstractly. The two attributes `state` and `drawn` that we created serve two purposes:
 - On one hand they are **internal variables** crucial for keeping track of the state of our object. They are used by the other methods of the object and are necessary for it to work properly.
 - On the other hand they can also be **accessed directly by the user** to see the current state (e.g. see what cards have been drawn).

This is a bit problematic. To better understand this think of the following example.

In [5]:
# Initialize the deck
deck = Deck()

print(f'{len(deck.stock) = }')

# Draw a card directly from the stock, without using the dedicated method
drawn_card = deck.stock[0]
deck.stock = deck.stock[1:]
print('Drawn card:', drawn_card)

# Reset the deck
deck.reset()

print('\nStock after reset')
print(f'{len(deck.stock) = }')

len(deck.stock) = 52
Drawn card: 3H

Stock after reset
len(deck.stock) = 51


By drawing from the stock direcly instead of using the dedicated `draw()` method, we broke some internal logic of the class and now after resetting the deck we have only 51 cards available instead of 52.

The issue here is that since `stock` and `draw` are sensitive to the object working properly they shouldn't also be accessed directly by the user. In python there is no way to enforce that the user can't access these (i.e. no such thing as a private or protected attribute). One way, which we discussed before would be to add an underscore before these attributes to indicate to the user that he shouldn't access these. This, however would require us to set up getter methods (e.g. `get_stock()`) so that the user has some way of viewing the state of the class.

In python there is a better way: using **properties**. Α property is a special kind of attribute that is accessed like a normal attribute, but is implemented through getters and setters.

*Note: properties are useful for a few purposes, we'll just use them, though, to obscure `stock` and `drawn`*

In [6]:
class Deck:
    _RANKS = tuple(range(2, 11)) + ('J', 'Q', 'K', 'A')
    _SUITS = ('D', 'H', 'C', 'S')
    _CARDS = tuple(str(v) + s for v in _RANKS for s in ('D', 'H', 'C', 'S'))
    _COLORS = {'D': 'R', 'H': 'R', 'C': 'B', 'S': 'B'}

    def __init__(self, num_packs=1):
        # Replace self.stock and self.drawn with self._stock and self._drawn
        self._stock = self.shuffle(self._CARDS * num_packs)
        self._drawn = []

    def draw(self, n=1):
        cards = [self._stock.pop(0) for _ in range(n)]
        self._drawn += cards
        return cards

    def reset(self):
        self._stock = self.shuffle(self._stock + self._drawn)
        self._drawn = []

    @property
    def stock(self):
        """
        This is the getter method of stock. Whenever we want to access the
        attribute self.stock, this method will be called.
        """
        return self._stock

    @property
    def drawn(self):
        """
        This is the getter method of drawn.
        """
        return self._drawn

    @staticmethod
    def shuffle(cards):
      # We also changed shuffle to be a static method since it has no need of
      # the object's current state to work
      return list(np.random.permutation(cards))

By defining the internal variables as properties, they can be accessed as they were before.

In [7]:
deck = Deck()

print('Initial stock:', deck.stock[:10])
print(f'{len(deck.stock) = }')

drawn_cards = deck.draw(3)
print('Drawn cards:', drawn_cards)

print('Stock after drawing cards:', deck.stock[:10])
print(f'{len(deck.stock) = }')

Initial stock: ['8C', '4H', 'AC', 'QS', '7S', '7C', 'KC', 'JC', '2C', '5S']
len(deck.stock) = 52
Drawn cards: ['8C', '4H', 'AC']
Stock after drawing cards: ['QS', '7S', '7C', 'KC', 'JC', '2C', '5S', '8S', '6C', '8H']
len(deck.stock) = 49


But if we try to modify this property it won't work, because we haven't defined the setter for this method.

In [8]:
# Uncomment to see the error

# deck.stock = deck.stock[1:]

AttributeError: can't set attribute 'stock'

## Add more interoperability with built-in operators and functions

Let's take a moment to think a bit about the `__init__` method that we defined previously. This is a special kind of method called a **dunder** (i.e. *double underscore*) method. It isn't the only one of its kind; by implementing dunder methods we **modify how the object behaves in certain situations**.

### Make objects work with `len`

Let's see an example. By convention in python, the `len` method works in all container objects and returns a notion of their size.

In [9]:
alist = [1, 2, 3, 4]
atuple = ('a', 'b', 'c')
adict = {'k1': 'v1', 'k2': 'v2'}

print(f'{len(alist) = }')
print(f'{len(atuple) = }')
print(f'{len(adict) = }')

len(alist) = 4
len(atuple) = 3
len(adict) = 2


But the `len` method does not only work for built-in objects. Not only that, each object defines this method in its own way.

In [10]:
import numpy as np
import pandas as pd

arr = np.arange(24).reshape(4, 2, 3)
ser = pd.Series([1, 2, 3])
df = pd.DataFrame({'a': [1, 2], 'b': [3, 4]})

print(f'{len(arr) = }')  # will return the size of the first axis
print(f'{len(ser) = }')  # will return the size
print(f'{len(df) = }')   # will return the number of rows

len(arr) = 4
len(ser) = 3
len(df) = 2


It seems appropriate that we should make the `len` method work for our objects as well. To do this we need to implement the special `__len__` dunder method.

In [11]:
class Deck:
    _RANKS = tuple(range(2, 11)) + ('J', 'Q', 'K', 'A')
    _SUITS = ('D', 'H', 'C', 'S')
    _CARDS = tuple(str(v) + s for v in _RANKS for s in ('D', 'H', 'C', 'S'))
    _COLORS = {'D': 'R', 'H': 'R', 'C': 'B', 'S': 'B'}

    def __init__(self, num_packs=1):
        self._stock = self.shuffle(self._CARDS * num_packs)
        self._drawn = []

    def draw(self, n=1):
        cards = [self._stock.pop(0) for _ in range(n)]
        self._drawn += cards
        return cards

    def reset(self):
        self._stock = self.shuffle(self._stock + self._drawn)
        self._drawn = []

    def __len__(self):
        """
        Define the len as the size of the stock
        """
        return len(self.stock)

    @property
    def stock(self):
        return self._stock

    @property
    def drawn(self):
        return self._drawn

    @staticmethod
    def shuffle(cards):
      return list(np.random.permutation(cards))

In [12]:
deck = Deck()

print(f'{len(deck) = }')

deck.draw(5)

print(f'{len(deck) = }')


len(deck) = 52
len(deck) = 47


### Add object's string representation

Most objects have a string representation (actually two to be precise).


In [13]:
arr  # repr

array([[[ 0,  1,  2],
        [ 3,  4,  5]],

       [[ 6,  7,  8],
        [ 9, 10, 11]],

       [[12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23]]])

In [14]:
print(arr)  # str

[[[ 0  1  2]
  [ 3  4  5]]

 [[ 6  7  8]
  [ 9 10 11]]

 [[12 13 14]
  [15 16 17]]

 [[18 19 20]
  [21 22 23]]]


These can be controlled by the following two dunder methods `__repr__` and `__str__`. Since the latter will default to the first if not defined, we'll only implement `__repr__`.

*Note: By convention this should include information that, when passed to the `eval()` function, would recreate an object with the same state. We won't follow the convention to avoid reimplementing the core functionality of our class..*

In [15]:
class Deck:
    _RANKS = tuple(range(2, 11)) + ('J', 'Q', 'K', 'A')
    _SUITS = ('D', 'H', 'C', 'S')
    _CARDS = tuple(str(v) + s for v in _RANKS for s in ('D', 'H', 'C', 'S'))
    _COLORS = {'D': 'R', 'H': 'R', 'C': 'B', 'S': 'B'}

    def __init__(self, num_packs=1):
        self._stock = self.shuffle(self._CARDS * num_packs)
        self._drawn = []

    def draw(self, n=1):
        cards = [self._stock.pop(0) for _ in range(n)]
        self._drawn += cards
        return cards

    def reset(self):
        self._stock = self.shuffle(self._stock + self._drawn)
        self._drawn = []

    def __len__(self):
        return len(self.stock)

    def __repr__(self):
        # simply return the string we want to be printed
        return f'Deck({len(self)})'

    @property
    def stock(self):
        return self._stock

    @property
    def drawn(self):
        return self._drawn

    @staticmethod
    def shuffle(cards):
      return list(np.random.permutation(cards))

In [16]:
deck = Deck()

print(deck)  # str (will fallback to repr since it's not defined)

deck.draw(8)

deck  # repr

Deck(52)


Deck(44)

### Accessing elements

Most container objects have some way for us to access their elements. The convention is that we will do this by using the square brackets operator.

The method of accessing the elements might change from object to object.

In [17]:
print(f'{alist[0] = }')      # by index
print(f'{alist[-1] = }')     # by negative index
print(f'{alist[1:3] = }')    # by slice

print(f'{adict["k2"] = }')   # by key

print(f'{arr[0, :, 1] = }')  # by index in multiple dims

mask = [False, True, True]
print(f'{ser[mask] = }')     # by binary mask

print(f'{ser[[0, 2]] = }')   # by multiple indexes


alist[0] = 1
alist[-1] = 4
alist[1:3] = [2, 3]
adict["k2"] = 'v2'
arr[0, :, 1] = array([1, 4])
ser[mask] = 1    2
2    3
dtype: int64
ser[[0, 2]] = 0    1
2    3
dtype: int64


All of this behavior can be defined in the `__getitem__` method.  

In [18]:
class Deck:
    _RANKS = tuple(range(2, 11)) + ('J', 'Q', 'K', 'A')
    _SUITS = ('D', 'H', 'C', 'S')
    _CARDS = tuple(str(v) + s for v in _RANKS for s in ('D', 'H', 'C', 'S'))
    _COLORS = {'D': 'R', 'H': 'R', 'C': 'B', 'S': 'B'}

    def __init__(self, num_packs=1):
        self._stock = self.shuffle(self._CARDS * num_packs)
        self._drawn = []

    def draw(self, n=1):
        cards = [self._stock.pop(0) for _ in range(n)]
        self._drawn += cards
        return cards

    def reset(self):
        self._stock = self.shuffle(self._stock + self._drawn)
        self._drawn = []

    def __getitem__(self, index):
        """
        Get the item(s) with the respective index (or slice) from the stock
        """
        return self.stock[index]

    def __len__(self):
        return len(self.stock)

    def __repr__(self):
        return f'Deck({len(self)})'

    @property
    def stock(self):
        return self._stock

    @property
    def drawn(self):
        return self._drawn

    @staticmethod
    def shuffle(cards):
      return list(np.random.permutation(cards))

In [19]:
deck = Deck()

# See first and last 10 elements
print(f'{deck.stock[:10] = }')
print(f'{deck.stock[-10:] = }')

print(f'{deck[2] = }')      # index
print(f'{deck[-5] = }')     # negative index

print(f'{deck[:5] = }')     # slice

deck.stock[:10] = ['QS', 'KH', '7C', 'AS', '8H', 'QC', '3S', 'KC', 'AC', '3D']
deck.stock[-10:] = ['7H', '10D', '5H', '9C', 'KD', '4S', 'JS', '6C', 'QH', '8D']
deck[2] = '7C'
deck[-5] = '4S'
deck[:5] = ['QS', 'KH', '7C', 'AS', '8H']


### Make the object iterable

Depending on what we want exactly we might need to do a few different things here. If we want to make our object a proper **iterator**, we need to follow the **iterator protocol** and define the `__next__` and `__iter__` methods. Arguably this would be the best way to proceed because it would allow our objects to do more stuff.

However, to showcase the simplicity of the matter, we'll stick with our object being an **iterable**, which in most cases suffices. An iterable is an object whose elements we can iterate over (i.e. through a `for` loop). How do we do this? We need to follow the **iterable protocol** and either implement the `__iter__` or the `__getitem__` method. Wait! The latter is already implemented! So we can simply iterate over our object without doing anything more? Let's see...

In [20]:
for card in deck:
    print(card)

QS
KH
7C
AS
8H
QC
3S
KC
AC
3D
10C
QD
KS
2H
4D
5S
6D
10H
7S
2S
8S
6S
5D
9S
JD
2C
3C
4C
3H
9D
9H
10S
8C
5C
JC
4H
AD
JH
AH
7D
2D
6H
7H
10D
5H
9C
KD
4S
JS
6C
QH
8D


### Missing sequence functionality

Our class is starting to become very functional. In fact it can be used anythere that a sequence is expected (just by implementing the `__len__` and `__getitem__` methods). Thus we can say that our class **follows the sequence protocol**.

We'll go one step further and add a few more methods to make it even closer to most python sequences.

These methods are:

- `__contains__`: need this to support membership checks (e.g. `'5D' in deck`).
- `__eq__`: need this to support equality checks (e.g. `deck1 == deck2`).
- remaining comparisons `__ne__` (`!=` opertor), `__lt__` (`<` operator), `__le__` (`<=` operator), `__gt__` (`>` operator), `__ge__` (`>=` operator).
- `count()`: count how many of one element are in the sequence.
- `index()`: return the index of an element.
- `__iter__`: return an interator of the sequence.
- `__reversed__`: return a reversed iterator of the sequence.
- `__bool__`: return the boolean evaluation of the sequence (not really necessary, but still nice to have). The general convension is that if the sequence is empty return `False`, in other cases return `True`.

*Note: Some of these are more important than others. We don't need to implement all of these every time.*

In [21]:
class Deck:

    _RANKS = tuple(range(2, 11)) + ('J', 'Q', 'K', 'A')
    _SUITS = ('D', 'H', 'C', 'S')
    _CARDS = tuple(str(v) + s for v in _RANKS for s in ('D', 'H', 'C', 'S'))
    _COLORS = {'D': 'R', 'H': 'R', 'C': 'B', 'S': 'B'}

    def __init__(self, num_packs=1):
        self._stock = self.shuffle(self._CARDS * num_packs)
        self._drawn = []

    def draw(self, n):
        cards = [self._stock.pop(0) for _ in range(n)]
        self._drawn += cards
        return cards

    def reset(self):
        self._stock = self.shuffle(self._stock + self._drawn)
        self._drawn = []

    def count(self, card):
        return self.stock.count(card)

    def index(self, card):
        return self.stock.index(card)

    def __getitem__(self, index):
        return self.stock[index]

    def __repr__(self):
        return f'Deck({len(self)})'

    def __len__(self):
        return len(self.stock)

    def __contains__(self, card):
        return card in self.stock

    def __eq__(self, other_deck):
        return len(self) == len(other_deck) and all([c1 == c2 for c1, c2
                                                 in zip(self, other_deck)])

    def __ne__(self, other_deck):
        return not (self == other_deck)

    def __gt__(self, other_deck):
        return len(self) > len(other_deck)

    def __lt__(self, other_deck):
        return len(self) < len(other_deck)

    def __ge__(self, other_deck):
        return len(self) >= len(other_deck)

    def __le__(self, other_deck):
        return len(self) <= len(other_deck)

    def __iter__(self):
        return iter(self.stock)

    def __reversed__(self):
        return reversed(self.stock)

    def __bool__(self):
        return bool(self.stock)

    @property
    def stock(self):
        return self._stock

    @property
    def drawn(self):
        return self._drawn

    @staticmethod
    def shuffle(cards):
      return list(np.random.permutation(cards))

In [22]:
deck1 = Deck(num_packs=3)
deck2 = Deck(num_packs=2)

# repr and len
print(f'{deck1 = }')
print(f'{deck2 = }')

# comparisons
print(f'{deck1 == deck2 = }')
print(f'{deck1 != deck2 = }')
print(f'{deck1 > deck2  = }')
print(f'{deck1 < deck2  = }')
print(f'{deck1 >= deck2 = }')
print(f'{deck1 <= deck2 = }')

# membership
print(f'{"5C" in deck1 = }')
print(f'{"12C" in deck1 = }')

# count
print(f'{deck1.count("5C") = }')

# index method and accessing elements
print(f'{deck1[deck1.index("5C")] == "5C" = }')

# iterable and slicing
first_5 = [card for card in deck1[:5]]
draw_5 = deck1.draw(5)
assert first_5 == draw_5

deck1 = Deck(156)
deck2 = Deck(104)
deck1 == deck2 = False
deck1 != deck2 = True
deck1 > deck2  = True
deck1 < deck2  = False
deck1 >= deck2 = True
deck1 <= deck2 = False
"5C" in deck1 = True
"12C" in deck1 = False
deck1.count("5C") = 3
deck1[deck1.index("5C")] == "5C" = True


### Implement conversion from and to binary

Another thing we can look at (though arguably not that useful) is to add a way to produce a binary representation of our object and subsequently to be able to instantite an object from its binary representation. This is interesting because it falls under the concept of **serialization**, i.e. to produce a representation of our object that can be shared and or stored.

We'll actually implement two seralization methods:
1. encode our object as a string
2. encode our object as bytes

In [23]:
class Deck:

    _RANKS = tuple(range(2, 11)) + ('J', 'Q', 'K', 'A')
    _SUITS = ('D', 'H', 'C', 'S')
    _CARDS = tuple(str(v) + s for v in _RANKS for s in ('D', 'H', 'C', 'S'))
    _COLORS = {'D': 'R', 'H': 'R', 'C': 'B', 'S': 'B'}

    def __init__(self, num_packs=1):
        self._stock = self.shuffle(self._CARDS * num_packs)
        self._drawn = []

    def draw(self, n):
        cards = [self._stock.pop(0) for _ in range(n)]
        self._drawn += cards
        return cards

    def reset(self):
        self._stock = self.shuffle(self._stock + self._drawn)
        self._drawn = []

    def count(self, card):
        return self.stock.count(card)

    def index(self, card):
        return self.stock.index(card)

    def tocardstr(self):
        """
        Encode the deck's state as a string.

        The string consists of two components (one for the stock and one for
        the drawn), which are delimited by '|'. Each component includes all cards
        in its respective list delimited by ','

        e.g.
        '2S,2C,2D,2H,3S,3C,...,JS|JC,JD,...,AH'
        |<------- stock -------->|<---drawn--->|
        """

        return ','.join(deck.stock) + '|' + ','.join(deck.drawn)

    def __getitem__(self, index):
        return self.stock[index]

    def __repr__(self):
        return f'Deck({len(self)})'

    def __len__(self):
        return len(self.stock)

    def __contains__(self, card):
        return card in self.stock

    def __eq__(self, other_deck):
        return len(self) == len(other_deck) and all([c1 == c2 for c1, c2
                                                 in zip(self, other_deck)])

    def __ne__(self, other_deck):
        return not (self == other_deck)

    def __gt__(self, other_deck):
        return len(self) > len(other_deck)

    def __lt__(self, other_deck):
        return len(self) < len(other_deck)

    def __ge__(self, other_deck):
        return len(self) >= len(other_deck)

    def __le__(self, other_deck):
        return len(self) <= len(other_deck)

    def __iter__(self):
        return iter(self.stock)

    def __reversed__(self):
        return reversed(self.stock)

    def __bytes__(self):
        """
        Produce a binary representation of the object.
        """
        return self.tocardstr().encode('utf-8')

    @classmethod
    def frombytes(cls, bytestr):
      """
      Instantiate an object from its binary representation
      """
      return cls.fromcardstr(bytestr.decode('utf-8'))


    @classmethod
    def fromcardstr(cls, cardstr):
        """
        Instantiate a new deck object from a cardstr
        """

        # isolate stock and drawn components of cardstr
        stockstr, drawnstr = cardstr.split('|')

        # instantiate new deck object
        deck = cls()

        # change state of new object to match cardstr
        deck._stock = stockstr.split(',') if stockstr else []
        deck._drawn = drawnstr.split(',') if drawnstr else []

        return deck

    @property
    def stock(self):
        return self._stock

    @property
    def drawn(self):
        return self._drawn

    @staticmethod
    def shuffle(cards):
      return list(np.random.permutation(cards))

In [24]:
# create a new deck object
deck = Deck()
deck.draw(10)

# get its binary representation
bytestr = bytes(deck)

print(f'{bytestr = }')

# instantiate a new deck object from the previous
deck2 = Deck.frombytes(bytestr)

# make sure that these two are equivalent
assert deck == deck2

bytestr = b'4C,6D,KC,KH,10H,KD,QH,7H,4H,AD,8C,9S,9D,10D,4D,6C,4S,6S,5C,QS,2D,5D,3D,3H,8D,5S,AC,3C,2H,10S,7S,QC,AH,QD,2S,JH,7C,8S,7D,JC,5H,KS|6H,2C,9C,9H,AS,3S,10C,8H,JS,JD'


## Discussion

### Other dunder methods

This tutorial wasn't meant as a full description of python's data model and the class dunder methods; instead the goal was to show the idea of making our objects more interoperable via implementing these methods.

As you can imagine there are several other dunder methods:
- some that make our objects work with the built in numeric opeations: `__add__` for `+`, `__sub__` for `-`, etc.
- for string formatting, i.e. `__str__` and `__format__`.
- for defining context managers, i.e. `__enter__` and `__exit__`.
- for controlling instance creatinion and deletion, i.e. `__new__` and `__del__`.
- for controlling attribute access: `__getattr__`, `__getattribute__`, etc.
- for defining descriotors: `__get__`, `__set__`, `__del__`.
- and many many more.

For a full list of python dunder methods you can read [the official documetation](https://docs.python.org/3/reference/datamodel.html#special-method-names).

### Dataclasses

The goal of this tutorial was to get a glimpse of the python data model and how custom classes can be made to use the language's operators and functions.

In some cases, we can use dataclasses to make our life a lot easier. These are special classes that essentially autogenerate the code necessary for the `__init__`, `__repr__`,  `__eq__`, `__ne__`, `__lt__`, `__le__`, `__gt__`, and `__ge__` methods. These are best used for simple classes whose primary function is to store data.

### Protocols and Duck Typing

Throughout this tutorial we've seen several cases the word **protocol** appear. But what exactly is a protocol? In the context of object-oriented programming, a protocol is an **informal interface**, defined only in documentation and not in code.

What does this mean in practice? In the `Deck` example, we mentioned that by implementing the `__len__` and `__getitem__` methods our class follows the sequence protocol and thus can be used anywhere a sequence is expected in python.

The general principle behind this is called **duck typing**. Duck typing is a programming concept that focuses on the **behavior** of an object rather than its **type**. The idea behind duck typing is inspired by the saying,

> If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck.

In other words, if an object behaves like a certain type, it is treated as an instance of that type, regardless of its actual class or type declaration. Thus in python, to make our objects interoperable with the language's built-ins, we don't need to subclass from some special type; rather we simply need to define some functionality.

This is upheld in the Python documentation, where for example certain functions will expect *a file-like object* or *an iterable* (instead of subclass of X base function).

Additionally, for some primative classes, the `isinstance` check will return true if a class fulfulls the respective protocol.

In [25]:
from collections import abc

class SizedMock:
    def __len__(self):
        return 12


class IterableMock:
    def __iter__(self):
        pass


class ContainerMock:
    def __contains__(self):
        pass

In [26]:
siz = SizedMock()
it = IterableMock()
con = ContainerMock()

print(f'{isinstance(siz, abc.Sized)}')
print(f'{isinstance(it, abc.Iterable)}')
print(f'{isinstance(con, abc.Container)}')

True
True
True


Note that in the examples above, the `isinstance` check returned `True`, even though the objects **are not a subclass** of the respective base classes!